In [1]:
import pandas as pd
import numpy as np
import math
import os
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
import Powerset as ps
from gensim import corpora, models, similarities
import os
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import time
import warnings
import random
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
df = pd.read_csv("metadata_reports_noduplicates_with_industry.csv")

df['industry'] = df['ggroup']
df['quarter_year'] = df["year"].astype("str") + " q" + df["quarter"].astype("str")

df2 = df.dropna(subset=['industry']).reset_index(drop=True)
df2['industry-quarter'] = list(zip(df2.industry, df2.quarter_year))
list_industries_quarter = df2.groupby('industry-quarter').count().reset_index()['industry-quarter'].tolist()

In [80]:
df2.count()['Available'] # Total number of reports

86867

In [81]:
temp=df2.groupby(['TICKER','Company Name']).count()['Available'].reset_index()
temp2=temp.groupby(['TICKER']).apply(lambda x: x.sort_values(['Available'], ascending=False))
del temp2['TICKER']
temp2=temp2.reset_index()
temp3=temp2.groupby('TICKER').first().reset_index()

In [95]:
temp3

,TICKER,level_1,Company Name,Available
0,AAL,9,American Airlines Group Inc,221
1,AAP,35,Advance Auto Parts Inc,190
2,AAPL,65,Apple Inc,484
3,ABBV,258,Abbvie Inc,254
4,ABC,345,AmerisourceBergen Corp,181
...,...,...,...,...
425,XRX,6950,Xerox Holdings Corp,35
426,YUM,6962,Yum! Brands Inc,163
427,ZBH,6969,Zimmer Biomet Holdings Inc,208
428,ZBRA,6973,Zebra Technologies Corp,125


In [96]:
temp3['Available'].sum() # Total number of reports

71686

In [97]:
temp3['TICKER_Comp']=temp3['TICKER']+'_'+temp3['Company Name']
list_sample=temp3['TICKER_Comp'].tolist()

In [98]:
df2['TICKER_Comp']=df2['TICKER']+'_'+df2['Company Name']

In [99]:
# select only relevant rows
df3=df2[df2['TICKER_Comp'].apply(lambda x: x in list_sample)]

In [100]:
df3.count()

Available                71686
Date                     71686
Doc Grp                  71686
Info                     71686
Prim. Ticker             71322
Second. Ticker            5267
RI                       69017
Title                    71686
Pages                    71676
Price                    71684
Contributor              71686
Analyst                  67883
Estimate Rating          45025
Recommendation Rating    36397
Company Name             71686
Country                  71646
Industry                 71683
Relevance                    0
DCN                      71686
TICKER                   71686
PrimaryTicker            71686
SecondaryTicker          71686
Match_Ticker             71686
GenderAnalyst            67883
year                     71686
quarter                  71686
PERMNO                   71686
gsector                  71686
ggroup                   71686
industry                 71686
quarter_year             71686
industry-quarter         71686
TICKER_C

In [94]:
df[df['industry'].isna()].groupby(['TICKER','Company Name']).count().to_csv('NonMatched_Industry.csv')

In [103]:
no_industry=df[df['industry'].isna()].groupby(['TICKER','Company Name']).count()

In [104]:
temp=df[df['industry'].isna()].groupby(['TICKER','Company Name']).count()['Available'].reset_index()
temp2=temp.groupby(['TICKER']).apply(lambda x: x.sort_values(['Available'], ascending=False))
del temp2['TICKER']
temp2=temp2.reset_index()
temp3=temp2.groupby('TICKER').first().reset_index()

In [112]:
len(temp3['TICKER'].tolist())

79

,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,Price,...,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup,quarter_year,industry-quarter,TICKER_Comp
industry,,,,,,,,,,,,,,,,,,,,,
1010.0,5282,5282,5282,5282,5121,1183,4705,5282,5279,5282,...,5282,4924,5282,5282,5282,5282,5282,5282,5282,5282
1510.0,3833,3833,3833,3833,3722,786,3557,3833,3832,3833,...,3833,3649,3833,3833,3833,3833,3833,3833,3833,3831
2010.0,6025,6025,6025,6025,5981,1014,5797,6025,6024,6025,...,6025,5726,6025,6025,6025,6025,6025,6025,6025,6023
2020.0,1176,1176,1176,1176,1162,319,1093,1176,1176,1176,...,1176,1109,1176,1176,1176,1176,1176,1176,1176,1176
2030.0,2983,2983,2983,2983,2953,665,2828,2983,2983,2983,...,2983,2920,2983,2983,2983,2983,2983,2983,2983,2983
2510.0,917,917,917,917,869,369,688,917,917,917,...,917,881,917,917,917,917,917,917,917,915
2520.0,3505,3505,3505,3505,3482,641,3393,3505,3500,3505,...,3505,3290,3505,3505,3505,3505,3505,3505,3505,3505
2530.0,2929,2929,2929,2929,2881,807,2805,2929,2929,2929,...,2929,2679,2929,2929,2929,2929,2929,2929,2929,2928
2550.0,7388,7388,7388,7388,7239,2468,6760,7388,7387,7386,...,7388,6903,7388,7388,7388,7388,7388,7388,7388,7387


In [149]:
matchdata=pd.read_csv('.\\IndustryMatching\\missing_industry_results_v2.csv')

In [151]:
missing_tickers=matchdata.groupby(['tic','ggroup']).count().reset_index()[['tic','ggroup']]
missing_tickers=missing_tickers.rename(columns={'tic':"TICKER"})

def replace_tickers(tick):
    
    if tick=='BRK.B':
        tick='BRKa'
    elif tick=='EL':
        tick='EI'
    elif tick=='FOXA':
        tick='FOX'
    elif tick=='HPE':
        tick='HPE (1)'
    elif tick=='BF.B':
        tick="BFb"
    elif tick=='VTRS': # merger here
        tick='MYL'
    
        
    return tick
    
missing_tickers['TICKER']=missing_tickers['TICKER'].apply(lambda x: replace_tickers(x))

In [152]:
set_found=set(missing_tickers['TICKER'].to_list())
set_missing=set(temp3['TICKER'].tolist())

In [153]:
set_found-set_missing

set()

In [156]:
missing_tickers

,TICKER,ggroup
0,ACN,4510
1,AIV,6010
2,ALLE,2010
3,AMCR,1510
4,AMT,6010
...,...,...
73,VTR,6010
74,MYL,3520
75,WELL,6010
76,WLTW,4030


In [158]:
df = pd.read_csv("metadata_reports_noduplicates_with_industry.csv")

In [168]:
def missing_replace(x):
    if x=='ETN':
        return 2010
    else:
        return missing_tickers[missing_tickers['TICKER']==x]['ggroup'].mean()

In [179]:
mask=(df['ggroup'].apply(lambda x: np.isnan(x)))
df.loc[mask,'ggroup']=df['TICKER'].apply(lambda x: missing_replace(x))

In [180]:
df

,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,Price,...,TICKER,PrimaryTicker,SecondaryTicker,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup
0,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Increase, TargetPrice Increase","3Q18 Recap: Maintaining Our Rating, Raising Ou...",9.0,92.0,...,AAL,['AAL.OQ'],['nan'],1,male,2018,4,21020.0,20.0,2030.0
1,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,Morningstar | Raising Our FVE After American S...,13.0,69.0,...,AAL,['AAL.OQ'],['nan'],1,male,2018,4,21020.0,20.0,2030.0
2,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, TargetPrice Increase",Morningstar | Rising Fuel Costs Raise American...,25.0,195.5,...,AAL,['AAL.OQ'],['nan'],1,male,2018,4,21020.0,20.0,2030.0
3,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,3Q18 Recap; Strategic Initiatives Aimed At Dri...,5.0,80.0,...,AAL,['AAL.OQ'],['nan'],1,male,2018,4,21020.0,20.0,2030.0
4,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, Recommendation Decrease, Ta...",American Airlines Group Inc.,9.0,69.0,...,AAL,['AAL.OQ'],['nan'],1,male,2018,4,21020.0,20.0,2030.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98729,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation - 4Q Core EPS $0.80 - So...,12.0,92.0,...,ZION,"['ZIONP.OQ', 'ZION.OQ']",['nan'],1,male,2018,1,84129.0,40.0,4010.0
98730,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Solid Fees Offset Slightly Smaller Balance She...,7.0,100.0,...,ZION,"['ZIONP.OQ', 'ZION.OQ']",['nan'],1,NaN,2018,1,84129.0,40.0,4010.0
98731,2018-02-05,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,4Q17 - Post Conference Call Model Update,7.0,69.0,...,ZION,"['ZIONP.OQ', 'ZION.OQ']",['nan'],1,male,2018,1,84129.0,40.0,4010.0
98732,2018-01-17,2018-01-10,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation,9.0,69.0,...,ZION,"['ZIONP.OQ', 'ZION.OQ']",['nan'],1,male,2018,1,84129.0,40.0,4010.0


In [181]:
df[df['ggroup'].apply(lambda x: np.isnan(x))]

,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,Price,...,TICKER,PrimaryTicker,SecondaryTicker,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup


In [182]:
df.to_csv("metadata_reports_noduplicates_full_industry.csv")

In [185]:
df.groupby('ggroup').count()['Available'].sum()

98734

In [186]:
df_temp = pd.read_csv("metadata_reports_noduplicates_with_industry.csv")


In [187]:
df_temp.groupby('ggroup').count()['Available'].sum()

86867

In [189]:
df_temp.count()

Available                98734
Date                     98734
Doc Grp                  98734
Info                     98734
Prim. Ticker             96579
Second. Ticker           22790
RI                       90483
Title                    98734
Pages                    98700
Price                    98732
Contributor              98734
Analyst                  93037
Estimate Rating          63452
Recommendation Rating    53891
Company Name             98706
Country                  98661
Industry                 98731
Relevance                    0
DCN                      98734
TICKER                   98734
PrimaryTicker            98734
SecondaryTicker          98734
Match_Ticker             98734
GenderAnalyst            93037
year                     98734
quarter                  98734
PERMNO                   86908
gsector                  86867
ggroup                   86867
dtype: int64